# MTA New York City Subway Average Weekday Ridership 2014 - 2019

Data visualization of ridership activity for all 473 stations in the New York City subway system prior to the COVID-19 pandemic. A special thanks to Colin Patrick Reid for his contribution of the code template in his article "Exploring and Visualizing Chicago Transit data using pandas and Bokeh — Part II (intro to Bokeh)" in Towards Data Science (https://towardsdatascience.com/exploring-and-visualizing-chicago-transit-data-using-pandas-and-bokeh-part-ii-intro-to-bokeh-5dca6c5ced10). MTA subway ridership by station data can be found here: https://new.mta.info/agency/new-york-city-transit/subway-bus-ridership-2019. The geospatial coordinates, or geometry, of the subway stations can be found here: https://data.cityofnewyork.us/Transportation/Subway-Stations/arq3-7z49. The linestring geometry for the subway lines can be found here: https://data.cityofnewyork.us/Transportation/Subway-Lines/3qz8-muuu. It is suggested that the files for the stations and lines be downloaded as shapefiles.

Part 1: Curing the NYC Subway average weekday ridership by station data.

First, we'll import the packages that we're going to use. 

In [1]:
#Importing packages
import pandas as pd
import numpy as np
#from bokeh.io import show, output_notebook
from bokeh.plotting import figure, show, output_notebook
#from bokeh.tile_providers import CARTODBPOSITRON
from bokeh.tile_providers import get_provider, Vendors, CARTODBPOSITRON
from pandas import Series, DataFrame


Make sure the Python session is set to the appropriate working drive. Next, we'll import the subway station ridership data as a dataframe and name it "ridership_df".

In [3]:
#Import ridership data
ridership_df = pd.read_csv('MTA_Subway_Weekday_Ridership_2019 - Sheet1.csv')
ridership_df.head()


Station_ID      Station_Name           Line       Boro    2014    2015  \
0           1          Astor Pl              6  Manhattan  17,803  17,274   
1           2          Canal St  J,N,Q,R,W,Z,6  Manhattan  44,362  44,409   
2           3           50th St              1  Manhattan  27,251  26,927   
3           4         Bergen St            2,3   Brooklyn   3,923   3,823   
4           5  Pennsylvania Ave              3   Brooklyn   5,718   6,997   

     2016    2017    2018    2019 2018-2019 Change Unnamed: 11  2019 Rank  
0  16,806  16,377  16,031  17,180           +1,149       +7.2%       84.0  
1  45,030  47,601  47,034  47,629             +595       +1.3%       16.0  
2  26,362  25,932  26,268  26,712             +444       +1.7%       39.0  
3   3,884   3,853   3,653   3,857             +204       +5.6%      334.0  
4   2,485   3,503   4,331   4,493             +162       +3.8%      310.0

Checking the data types for the variables in ridership_df. 

In [4]:
#ridership_df = ridership_df.dropna()
ridership_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 473 entries, 0 to 472
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Station_ID        473 non-null    int64  
 1   Station_Name      473 non-null    object 
 2   Line              432 non-null    object 
 3   Boro              432 non-null    object 
 4   2014              428 non-null    object 
 5   2015              429 non-null    object 
 6   2016              429 non-null    object 
 7   2017              432 non-null    object 
 8   2018              432 non-null    object 
 9   2019              432 non-null    object 
 10  2018-2019 Change  432 non-null    object 
 11  Unnamed: 11       432 non-null    object 
 12  2019 Rank         432 non-null    float64
dtypes: float64(1), int64(1), object(11)
memory usage: 48.2+ KB


All of the variables, except for one, of the object or string data type. We need to  conduct some data cleaning (i.e., removing the thousands separator: ","), as well as renaming the year variables (or columns). We rename the variables, because when we reference a year, python will think its an integer.

In [5]:
ridership_df = ridership_df.rename(columns={'Unnamed: 11':'Pct_chg', 
                                            '2014':'y_2014',
                                            '2015':'y_2015',
                                            '2016':'y_2016',
                                            '2017':'y_2017',
                                            '2018':'y_2018',
                                            '2019':'y_2019',
                                            'Line':'Subway_lines',
                                            '2018-2019 Change':'Chg',
                                            '2019 Rank':'Rank_2019'
                                           })
ridership_df.head()

Station_ID      Station_Name   Subway_lines       Boro  y_2014  y_2015  \
0           1          Astor Pl              6  Manhattan  17,803  17,274   
1           2          Canal St  J,N,Q,R,W,Z,6  Manhattan  44,362  44,409   
2           3           50th St              1  Manhattan  27,251  26,927   
3           4         Bergen St            2,3   Brooklyn   3,923   3,823   
4           5  Pennsylvania Ave              3   Brooklyn   5,718   6,997   

   y_2016  y_2017  y_2018  y_2019     Chg Pct_chg  Rank_2019  
0  16,806  16,377  16,031  17,180  +1,149   +7.2%       84.0  
1  45,030  47,601  47,034  47,629    +595   +1.3%       16.0  
2  26,362  25,932  26,268  26,712    +444   +1.7%       39.0  
3   3,884   3,853   3,653   3,857    +204   +5.6%      334.0  
4   2,485   3,503   4,331   4,493    +162   +3.8%      310.0

In [6]:
ridership_df['y_2014'] = ridership_df.y_2014.str.replace(',', '')
ridership_df['y_2015'] = ridership_df.y_2015.str.replace(',', '')
ridership_df['y_2016'] = ridership_df.y_2016.str.replace(',', '')
ridership_df['y_2017'] = ridership_df.y_2017.str.replace(',', '')
ridership_df['y_2018'] = ridership_df.y_2018.str.replace(',', '')
ridership_df['y_2019'] = ridership_df.y_2019.str.replace(',', '')

ridership_df.head()

Station_ID      Station_Name   Subway_lines       Boro y_2014 y_2015  \
0           1          Astor Pl              6  Manhattan  17803  17274   
1           2          Canal St  J,N,Q,R,W,Z,6  Manhattan  44362  44409   
2           3           50th St              1  Manhattan  27251  26927   
3           4         Bergen St            2,3   Brooklyn   3923   3823   
4           5  Pennsylvania Ave              3   Brooklyn   5718   6997   

  y_2016 y_2017 y_2018 y_2019     Chg Pct_chg  Rank_2019  
0  16806  16377  16031  17180  +1,149   +7.2%       84.0  
1  45030  47601  47034  47629    +595   +1.3%       16.0  
2  26362  25932  26268  26712    +444   +1.7%       39.0  
3   3884   3853   3653   3857    +204   +5.6%      334.0  
4   2485   3503   4331   4493    +162   +3.8%      310.0

In [7]:
ridership_df

Station_ID                 Station_Name   Subway_lines       Boro y_2014  \
0             1                     Astor Pl              6  Manhattan  17803   
1             2                     Canal St  J,N,Q,R,W,Z,6  Manhattan  44362   
2             3                      50th St              1  Manhattan  27251   
3             4                    Bergen St            2,3   Brooklyn   3923   
4             5             Pennsylvania Ave              3   Brooklyn   5718   
..          ...                          ...            ...        ...    ...   
468         469  Coney Island - Stillwell Av        D,F,N,Q   Brooklyn  13734   
469         470       34th St - Hudson Yards              7  Manhattan    NaN   
470         641                      72nd St              Q  Manhattan    NaN   
471         642                      86th St              Q  Manhattan    NaN   
472         643                      96th St              Q  Manhattan    NaN   

    y_2015 y_2016 y_2017 y_2018 y_2019     Chg Pct_chg  Rank_2019  
0    17274  16806  16377  16031  17180  +1,149   +7.2%       84.0  
1    44409  45030  47601  47034  47629    +595   +1.3%       16.0  
2    26927  26362  25932  26268  26712    +444   +1.7%       39.0  
3     3823   3884   3853   3653   3857    +204   +5.6%      334.0  
4     6997   2485   3503   4331   4493    +162   +3.8%      310.0  
..     ...    ...    ...    ...    ...     ...     ...        ...  
468  14073  14098  13542  13203  12819    -384   -2.9%      122.0  
469   2064   8507  10082  10789  18875  +8,086  +75.0%       74.0  
470    NaN    NaN  28145  30253  31585  +1,332   +4.4%       30.0  
471    NaN    NaN  23722  25455  26307    +852   +3.3%       42.0  
472    NaN    NaN  17150  19318  19704    +386   +2.0%       72.0  

[473 rows x 13 columns]

Filling the blank entries with a value of zero.

In [8]:
ridership_df.fillna(value=0, axis=0, inplace=True)
ridership_df.tail()

Station_ID                 Station_Name Subway_lines       Boro y_2014  \
468         469  Coney Island - Stillwell Av      D,F,N,Q   Brooklyn  13734   
469         470       34th St - Hudson Yards            7  Manhattan      0   
470         641                      72nd St            Q  Manhattan      0   
471         642                      86th St            Q  Manhattan      0   
472         643                      96th St            Q  Manhattan      0   

    y_2015 y_2016 y_2017 y_2018 y_2019     Chg Pct_chg  Rank_2019  
468  14073  14098  13542  13203  12819    -384   -2.9%      122.0  
469   2064   8507  10082  10789  18875  +8,086  +75.0%       74.0  
470      0      0  28145  30253  31585  +1,332   +4.4%       30.0  
471      0      0  23722  25455  26307    +852   +3.3%       42.0  
472      0      0  17150  19318  19704    +386   +2.0%       72.0

By examining the tail of the ridership_df there are 473 rows in the dataset. However, there are 432 stations in the NYC subway system. In the ridership_df, there are some stations that are part of the Staten Island Railroad system (SIRR), which is separate from the NYC subway system as none of the subway lines connect to the stations that are in the SIRR. To address this issue, we dropped the stations that do not have a subway line connection (namely the SIRR stations).

In [10]:
ridership_df = ridership_df[ridership_df['Subway_lines']!=0]

Next, we convert the ridership data from objects or strings to integers. Then, we check the data types to ensure that the conversion was done appropriately.

In [12]:
ridership_df['y_2014'] = ridership_df.y_2014.astype(int)
ridership_df['y_2015'] = ridership_df.y_2015.astype(int)
ridership_df['y_2016'] = ridership_df.y_2016.astype(int)
ridership_df['y_2017'] = ridership_df.y_2017.astype(int)
ridership_df['y_2018'] = ridership_df.y_2018.astype(int)
ridership_df['y_2019'] = ridership_df.y_2019.astype(int)
ridership_df['2019_Rank'] = ridership_df.Rank_2019.astype(int)

ridership_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 432 entries, 0 to 472
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Station_ID    432 non-null    int64  
 1   Station_Name  432 non-null    object 
 2   Subway_lines  432 non-null    object 
 3   Boro          432 non-null    object 
 4   y_2014        432 non-null    int64  
 5   y_2015        432 non-null    int64  
 6   y_2016        432 non-null    int64  
 7   y_2017        432 non-null    int64  
 8   y_2018        432 non-null    int64  
 9   y_2019        432 non-null    int64  
 10  Chg           432 non-null    object 
 11  Pct_chg       432 non-null    object 
 12  Rank_2019     432 non-null    float64
 13  2019_Rank     432 non-null    int64  
dtypes: float64(1), int64(8), object(5)
memory usage: 50.6+ KB


/var/folders/vg/vb3rtmr10v34g75s975q03m80000gn/T/ipykernel_10253/3403491567.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ridership_df['y_2014'] = ridership_df.y_2014.astype(int)
/var/folders/vg/vb3rtmr10v34g75s975q03m80000gn/T/ipykernel_10253/3403491567.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ridership_df['y_2015'] = ridership_df.y_2015.astype(int)
/var/folders/vg/vb3rtmr10v34g75s975q03m80000gn/T/ipykernel_10253/3403491567.py:3: SettingWithCopyWarning: 
A value is trying to be set on a co

The information of the dataframe shows that we now have 432 stations, which is the exact number of stations in the NYC subway system, and that the data types have been converted appropriately.

Part 2: Working with the subway stations shapefile

We have completed our data curing of the ridership_df data. Next, we work on the shapefiles for the subway stations and lines. To do this, we import Geopandas (Please note, to use Geopandas, you may need to create a new environment. Instructions on how to do this can be found here: https://geopandas.org/en/stable/getting_started/install.html).

In [14]:
#Read in the MTA subway stations
import geopandas as gpd


In [16]:
stations_shapefiles = 'geo_export_10d03060-d569-40de-b528-c7b95b31aadd.shp'

Using Geopandas to read in the shapefiles and convert into a Geopandas DataFrame, naming it "stations_gdf".

In [17]:
stations_gdf = gpd.read_file(stations_shapefiles)
stations_gdf.head()

line              name  \
0  4-6-6 Express          Astor Pl   
1  4-6-6 Express          Canal St   
2            1-2           50th St   
3          2-3-4         Bergen St   
4            3-4  Pennsylvania Ave   

                                               notes  objectid  \
0  4 nights, 6-all times, 6 Express-weekdays AM s...       1.0   
1  4 nights, 6-all times, 6 Express-weekdays AM s...       2.0   
2                              1-all times, 2-nights       3.0   
3           4-nights, 3-all other times, 2-all times       4.0   
4                        4-nights, 3-all other times       5.0   

                                 url                    geometry  
0  http://web.mta.info/nyct/service/  POINT (-73.99107 40.73005)  
1  http://web.mta.info/nyct/service/  POINT (-74.00019 40.71880)  
2  http://web.mta.info/nyct/service/  POINT (-73.98385 40.76173)  
3  http://web.mta.info/nyct/service/  POINT (-73.97500 40.68086)  
4  http://web.mta.info/nyct/service/  POINT (-73.89489 40.66471)

Next, we merge the stations_gdf with ridership_df. We merge left dataset (stations_gdf) using "objectid" with the right dataset (ridership_df) using "Station_ID". We will call the merged dataset "merged".

In [18]:
#merging the 2 datasets
merged = stations_gdf.merge(ridership_df, left_on='objectid', right_on='Station_ID')

merged.head()

line              name  \
0  4-6-6 Express          Astor Pl   
1  4-6-6 Express          Canal St   
2            1-2           50th St   
3          2-3-4         Bergen St   
4            3-4  Pennsylvania Ave   

                                               notes  objectid  \
0  4 nights, 6-all times, 6 Express-weekdays AM s...       1.0   
1  4 nights, 6-all times, 6 Express-weekdays AM s...       2.0   
2                              1-all times, 2-nights       3.0   
3           4-nights, 3-all other times, 2-all times       4.0   
4                        4-nights, 3-all other times       5.0   

                                 url                    geometry  Station_ID  \
0  http://web.mta.info/nyct/service/  POINT (-73.99107 40.73005)           1   
1  http://web.mta.info/nyct/service/  POINT (-74.00019 40.71880)           2   
2  http://web.mta.info/nyct/service/  POINT (-73.98385 40.76173)           3   
3  http://web.mta.info/nyct/service/  POINT (-73.97500 40.68086)           4   
4  http://web.mta.info/nyct/service/  POINT (-73.89489 40.66471)           5   

       Station_Name   Subway_lines       Boro  y_2014  y_2015  y_2016  y_2017  \
0          Astor Pl              6  Manhattan   17803   17274   16806   16377   
1          Canal St  J,N,Q,R,W,Z,6  Manhattan   44362   44409   45030   47601   
2           50th St              1  Manhattan   27251   26927   26362   25932   
3         Bergen St            2,3   Brooklyn    3923    3823    3884    3853   
4  Pennsylvania Ave              3   Brooklyn    5718    6997    2485    3503   

   y_2018  y_2019     Chg Pct_chg  Rank_2019  2019_Rank  
0   16031   17180  +1,149   +7.2%       84.0         84  
1   47034   47629    +595   +1.3%       16.0         16  
2   26268   26712    +444   +1.7%       39.0         39  
3    3653    3857    +204   +5.6%      334.0        334  
4    4331    4493    +162   +3.8%      310.0        310

In [19]:
merged.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 432 entries, 0 to 431
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   line          432 non-null    object  
 1   name          432 non-null    object  
 2   notes         432 non-null    object  
 3   objectid      432 non-null    float64 
 4   url           432 non-null    object  
 5   geometry      432 non-null    geometry
 6   Station_ID    432 non-null    int64   
 7   Station_Name  432 non-null    object  
 8   Subway_lines  432 non-null    object  
 9   Boro          432 non-null    object  
 10  y_2014        432 non-null    int64   
 11  y_2015        432 non-null    int64   
 12  y_2016        432 non-null    int64   
 13  y_2017        432 non-null    int64   
 14  y_2018        432 non-null    int64   
 15  y_2019        432 non-null    int64   
 16  Chg           432 non-null    object  
 17  Pct_chg       432 non-null    object  
 18  Ra

As shown in merged.info(), we have some repetitive variables, such as line and Subway_lines. One of the easiest ways to address this issue, is to transform "merged" dataframe as a subset dataframe by including only the variables that we want to work with (dropping the repetitive variables).

In [20]:
merged = merged[['Station_ID',
                 'Station_Name',
                 'Subway_lines',
                 'Boro', 
                 'y_2014',
                 'y_2015',
                 'y_2016',
                 'y_2017',
                 'y_2018',
                 'y_2019',
                 'Chg',
                 'Pct_chg',
                 'Rank_2019',
                 'geometry'
                ]]

merged.head()

Station_ID      Station_Name   Subway_lines       Boro  y_2014  y_2015  \
0           1          Astor Pl              6  Manhattan   17803   17274   
1           2          Canal St  J,N,Q,R,W,Z,6  Manhattan   44362   44409   
2           3           50th St              1  Manhattan   27251   26927   
3           4         Bergen St            2,3   Brooklyn    3923    3823   
4           5  Pennsylvania Ave              3   Brooklyn    5718    6997   

   y_2016  y_2017  y_2018  y_2019     Chg Pct_chg  Rank_2019  \
0   16806   16377   16031   17180  +1,149   +7.2%       84.0   
1   45030   47601   47034   47629    +595   +1.3%       16.0   
2   26362   25932   26268   26712    +444   +1.7%       39.0   
3    3884    3853    3653    3857    +204   +5.6%      334.0   
4    2485    3503    4331    4493    +162   +3.8%      310.0   

                     geometry  
0  POINT (-73.99107 40.73005)  
1  POINT (-74.00019 40.71880)  
2  POINT (-73.98385 40.76173)  
3  POINT (-73.97500 40.68086)  
4  POINT (-73.89489 40.66471)

Next, we have to conduct some data manipulation on the geometry.

First, we split the x and y (longitude and latitude) coordinate into two separate columns as Bokeh cannot the geometry read as constructed.

In [21]:
def getPointCoords(row, geom, coord_type):
    """Calculates coordinates ('x' or 'y') of a Point geometry"""
    if coord_type == 'x':
        return row[geom].x
    elif coord_type == 'y':
        return row[geom].y

In [22]:
# Calculate x coordinates
merged['x'] = merged.apply(getPointCoords, geom='geometry', coord_type='x', axis=1)

# Calculate y coordinates
merged['y'] = merged.apply(getPointCoords, geom='geometry', coord_type='y', axis=1)

# Let's see what we have now
merged.head()

Station_ID      Station_Name   Subway_lines       Boro  y_2014  y_2015  \
0           1          Astor Pl              6  Manhattan   17803   17274   
1           2          Canal St  J,N,Q,R,W,Z,6  Manhattan   44362   44409   
2           3           50th St              1  Manhattan   27251   26927   
3           4         Bergen St            2,3   Brooklyn    3923    3823   
4           5  Pennsylvania Ave              3   Brooklyn    5718    6997   

   y_2016  y_2017  y_2018  y_2019     Chg Pct_chg  Rank_2019  \
0   16806   16377   16031   17180  +1,149   +7.2%       84.0   
1   45030   47601   47034   47629    +595   +1.3%       16.0   
2   26362   25932   26268   26712    +444   +1.7%       39.0   
3    3884    3853    3653    3857    +204   +5.6%      334.0   
4    2485    3503    4331    4493    +162   +3.8%      310.0   

                     geometry          x          y  
0  POINT (-73.99107 40.73005) -73.991070  40.730054  
1  POINT (-74.00019 40.71880) -74.000193  40.718803  
2  POINT (-73.98385 40.76173) -73.983849  40.761728  
3  POINT (-73.97500 40.68086) -73.974999  40.680862  
4  POINT (-73.89489 40.66471) -73.894886  40.664714

In [23]:
# Make a copy and drop the geometry column
df = merged.drop('geometry', axis=1).copy()

This merged dataframe will be the main dataframe we will be working from, so we rename it "df".

In [24]:
df.head()

Station_ID      Station_Name   Subway_lines       Boro  y_2014  y_2015  \
0           1          Astor Pl              6  Manhattan   17803   17274   
1           2          Canal St  J,N,Q,R,W,Z,6  Manhattan   44362   44409   
2           3           50th St              1  Manhattan   27251   26927   
3           4         Bergen St            2,3   Brooklyn    3923    3823   
4           5  Pennsylvania Ave              3   Brooklyn    5718    6997   

   y_2016  y_2017  y_2018  y_2019     Chg Pct_chg  Rank_2019          x  \
0   16806   16377   16031   17180  +1,149   +7.2%       84.0 -73.991070   
1   45030   47601   47034   47629    +595   +1.3%       16.0 -74.000193   
2   26362   25932   26268   26712    +444   +1.7%       39.0 -73.983849   
3    3884    3853    3653    3857    +204   +5.6%      334.0 -73.974999   
4    2485    3503    4331    4493    +162   +3.8%      310.0 -73.894886   

           y  
0  40.730054  
1  40.718803  
2  40.761728  
3  40.680862  
4  40.664714

Next, we conduct string manipulation to re-construct the geometry and create a new column called "Location". This step is not really necessary but is a good code to follow if you need to reconstruct the geometry with x and y coordinates. Here, the Location is in "latitude, longitude" format. Please note, that the geometries in shapefiles are in a "longitude, latitude" format.

In [25]:
df['Location'] = "(" + df['y'].astype(str) + ", " + df['x'].astype(str) +")"

In [26]:
df.head()

Station_ID      Station_Name   Subway_lines       Boro  y_2014  y_2015  \
0           1          Astor Pl              6  Manhattan   17803   17274   
1           2          Canal St  J,N,Q,R,W,Z,6  Manhattan   44362   44409   
2           3           50th St              1  Manhattan   27251   26927   
3           4         Bergen St            2,3   Brooklyn    3923    3823   
4           5  Pennsylvania Ave              3   Brooklyn    5718    6997   

   y_2016  y_2017  y_2018  y_2019     Chg Pct_chg  Rank_2019          x  \
0   16806   16377   16031   17180  +1,149   +7.2%       84.0 -73.991070   
1   45030   47601   47034   47629    +595   +1.3%       16.0 -74.000193   
2   26362   25932   26268   26712    +444   +1.7%       39.0 -73.983849   
3    3884    3853    3653    3857    +204   +5.6%      334.0 -73.974999   
4    2485    3503    4331    4493    +162   +3.8%      310.0 -73.894886   

           y                                 Location  
0  40.730054  (40.73005400028978, -73.99106999861966)  
1  40.718803  (40.71880300107709, -74.00019299927328)  
2  40.761728  (40.76172799961419, -73.98384899986625)  
3  40.680862  (40.68086213682956, -73.97499915116808)  
4  40.664714  (40.66471445143568, -73.89488591154061)

We're going to use the mercator projection to map the geometries in Bokeh. Below, we execute the following lines of code to convert the x and y coordinates to mercator projection coordinates.

In [28]:
import math
from ast import literal_eval
def merc(Coords):
    Coordinates = literal_eval(Coords)
    lat = Coordinates[0]
    lon = Coordinates[1]
    
    r_major = 6378137.000
    x = r_major * math.radians(lon)
    scale = x/lon
    y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + 
        lat * (math.pi/180.0)/2.0)) * scale
    return (x, y)

In [29]:
merc('(40.730054, -73.991070)')

(-8236648.23564946, 4972605.8591247015)

In [30]:
df['coords_x'] = df['Location'].apply(lambda x: merc(x)[0])
df['coords_y'] = df['Location'].apply(lambda x: merc(x)[1])

In [31]:
df.head()

Station_ID      Station_Name   Subway_lines       Boro  y_2014  y_2015  \
0           1          Astor Pl              6  Manhattan   17803   17274   
1           2          Canal St  J,N,Q,R,W,Z,6  Manhattan   44362   44409   
2           3           50th St              1  Manhattan   27251   26927   
3           4         Bergen St            2,3   Brooklyn    3923    3823   
4           5  Pennsylvania Ave              3   Brooklyn    5718    6997   

   y_2016  y_2017  y_2018  y_2019     Chg Pct_chg  Rank_2019          x  \
0   16806   16377   16031   17180  +1,149   +7.2%       84.0 -73.991070   
1   45030   47601   47034   47629    +595   +1.3%       16.0 -74.000193   
2   26362   25932   26268   26712    +444   +1.7%       39.0 -73.983849   
3    3884    3853    3653    3857    +204   +5.6%      334.0 -73.974999   
4    2485    3503    4331    4493    +162   +3.8%      310.0 -73.894886   

           y                                 Location      coords_x  \
0  40.730054  (40.73005400028978, -73.99106999861966) -8.236648e+06   
1  40.718803  (40.71880300107709, -74.00019299927328) -8.237664e+06   
2  40.761728  (40.76172799961419, -73.98384899986625) -8.235844e+06   
3  40.680862  (40.68086213682956, -73.97499915116808) -8.234859e+06   
4  40.664714  (40.66471445143568, -73.89488591154061) -8.225941e+06   

       coords_y  
0  4.972606e+06  
1  4.970953e+06  
2  4.977260e+06  
3  4.965382e+06  
4  4.963012e+06

Now that we have our mercator projection coordinates, we can plot the location of the subway stations using those coordinates.

In [32]:
#p = figure(x_range=(-8237000, -8200000), y_range=(4980000, 4950000),
#           x_axis_type="mercator", y_axis_type="mercator")

#tile_provider = get_provider(Vendors.CARTODBPOSITRON)
#p.add_tile(tile_provider)
#show(p)

In [33]:
p = figure(x_axis_type="mercator", y_axis_type="mercator")

tile_provider = get_provider(Vendors.CARTODBPOSITRON)
p.add_tile(tile_provider)
#p.add_tile(CARTODBPOSITRON)

p.circle(x = 'coords_x',
         y = 'coords_y',
         source = df)
output_notebook()
show(p)

Loading BokehJS ...

As shown on the map above, we have the subway stations plotted. However, we want to visualize the ridership activity of the stations. Therefore, we incorporate the data from ridership_df into this map.

We import items from Bokeh's io, models, and palettes libraries to do this. A couple things worth mentioning. One, for any visualization created in Bokeh, the ColumnDataSource is a list of datapoints that you want to use in you visualization. 

Second, the HoverTool is an interactive feature that you can incorporate into your figure which allows the user to "hover" over a station and a dialogue box will pop up providing ridership data, as well the subway lines that service the station. 

Third, the ridership data is not normally or equally distributed among stations. For example, there several stations outside of Manhattan that have under in which the average weekday ridership is under 2,000; whereas, there are several stations in Manhattan such as Times Square, Grand Central - 42nd and Penn Station - 34th Street have average weekday ridership over 50,000. The ridership data is both top and bottom heavy. When you have data that have these characteristics, the color mapper used may only present station ridership in 2 or 3 colors, which doesn't give a good assessment on ridership activity by station. To address this, we import the EqHistColorMapper from the Bokeh models library. By using the EqHistColorMapper, we show ridership activity based on 2019 numbers in a more "normalized" orientation, as shown by the several various colored stations in the "Magma" palette that we've selected. 

In [ ]:
#import matplotlib as mpl

In [34]:
from bokeh.io import output_notebook, show, output_file
from bokeh.models import ColumnDataSource, HoverTool, EqHistColorMapper, ColorBar
from bokeh.palettes import brewer
from bokeh.palettes import Magma, Inferno, Plasma, Viridis

source = ColumnDataSource(data=dict(
                        x=list(df['coords_x']), 
                        y=list(df['coords_y']),
                        y_2014 = list(df['y_2014']),
                        y_2015 = list(df['y_2015']),
                        y_2016 = list(df['y_2016']),
                        y_2017 = list(df['y_2017']),
                        y_2018 = list(df['y_2018']),
                        y_2019 = list(df['y_2019']),
                        Station = list(df['Station_Name']),
                        Subway_Lines=list(df['Subway_lines']),
                        Change=list(df['Chg']),
                        Pct_chg_2019=list(df['Pct_chg']),
                        Rank_2019=list(df['Rank_2019'])))

hover = HoverTool(tooltips=[
    ("Station", "@Station"),
    ("Subway Lines", "@Subway_Lines"),
    ("2019", "@{y_2019}{0,0}"),
    ("2018", "@{y_2018}{0,0}"),
    ("2017", "@{y_2017}{0,0}"),
    ("2016", "@{y_2016}{0,0}"),
    ("2015", "@{y_2015}{0,0}"),
    ("2014", "@{y_2014}{0,0}"),
    ("Actual change from 2018", "@Change"),
    ("% change from 2018", "@Pct_chg_2019"),
    ("Rank", "@Rank_2019")
])

p = figure(title = 'Average Weekday Subway Ridership, 2019',
           x_axis_type="mercator",
           y_axis_type="mercator",
           tools=[hover, 'wheel_zoom']
          )

tile_provider = get_provider(Vendors.CARTODBPOSITRON)
p.add_tile(tile_provider)

#Define a sequential multi-hue color palette.
#palette = brewer['Plasma'][8]
#palette = mpl['Plasma'][4]
#palette = Plasma[4]
palette = Magma[11]

#Reverse color order so that dark purple/magenta is to signal high activity.
palette = palette[::-1]

#exp_cmap = LinearColorMapper(palette=palette,
#                            low = min(df['y_2019']),
#                            high = max(df['y_2019']))

exp_cmap = EqHistColorMapper(palette=palette, low=min(df['y_2019']), high=max(df['y_2019']))

p.circle(x='x',
         y='y',
         source=source,
         size=6,
         line_color="black",
         fill_color={"field":"y_2019", "transform":exp_cmap},
         alpha=1
        )

bar = ColorBar(color_mapper=exp_cmap, location=(0,0))
#bar.formatter = NumeralTickFormatter(format'0,0')
p.add_layout(bar, 'left')

output_file("NYC_Subway_Ridership_Map_2019.html")

show(p)

Part 3: Applying the subway lines to the map.

Again, we're going to use Geopandas to read in the shapefile for the subway lines. Unlike the geometry of the subway stations, which are points, the geometry of subway lines are linestrings. To make this visualization more engaging, we're going to color the lines to make it mimic the NYC subway map.


First, we convert the coordinates of the subway lines into mercator projection coordinates. The process to converting the coordinates of the linestrings into mercator projection coordinates involve one additional step, compared to converting coordinates for points. Unlike points, a linestring can have several pairs of coordinates. To transform these coordinates into mercator projection coordinates, we followed the guidance in this StackExchange post (https://gis.stackexchange.com/questions/346181/project-large-collection-of-shapely-object-to-different-crs). We're going to call the lines dataset "lines_merc_gdf".

In [ ]:
###Mercator conversion

In [46]:
lines_shapefiles = 'geo_export_5cda2676-7bd2-492e-b02b-e69dd962562f.shp'

In [47]:
lines_merc_gdf = gpd.read_file(lines_shapefiles)

#https://gis.stackexchange.com/questions/346181/project-large-collection-of-shapely-object-to-different-crs
merc_conv = lines_merc_gdf['geometry'].to_crs("epsg:3857")
lines_merc_gdf['merc_coords'] = merc_conv
#gdf_wgs84.to_file("output.geojson", driver="GeoJSON")

lines_merc_gdf.head()

id name  objectid rt_symbol    shape_len  \
0  2000393.0    G     753.0         G  2438.200249   
1  2000394.0    G     754.0         G  3872.834411   
2  2000469.0    Q     755.0         N  1843.366331   
3  2000294.0    M     756.0         B  1919.559203   
4  2000296.0    M     757.0         B  2385.698536   

                                 url  \
0  http://web.mta.info/nyct/service/   
1  http://web.mta.info/nyct/service/   
2  http://web.mta.info/nyct/service/   
3  http://web.mta.info/nyct/service/   
4  http://web.mta.info/nyct/service/   

                                            geometry  \
0  LINESTRING (-73.99488 40.68020, -73.99427 40.6...   
1  LINESTRING (-73.97958 40.65993, -73.97966 40.6...   
2  LINESTRING (-73.97586 40.57597, -73.97654 40.5...   
3  LINESTRING (-73.92414 40.75229, -73.92405 40.7...   
4  LINESTRING (-73.91345 40.75617, -73.90905 40.7...   

                                         merc_coords  
0  LINESTRING (-8237071.834 4965285.580, -8237004...  
1  LINESTRING (-8235368.666 4962310.184, -8235377...  
2  LINESTRING (-8234954.663 4949997.756, -8235031...  
3  LINESTRING (-8229198.018 4975873.004, -8229187...  
4  LINESTRING (-8228007.267 4976443.278, -8227517...

In [48]:
def getLineCoords(row, geom, coord_type):
    """Returns a list of coordinates ('x' or 'y') of a LineString geometry"""
    if coord_type == 'x':
        return list( row[geom].coords.xy[0] )
    elif coord_type == 'y':
        return list( row[geom].coords.xy[1] )


In [49]:
# Calculate x coordinates of the line
lines_merc_gdf['x'] = lines_merc_gdf.apply(getLineCoords, geom='merc_coords', coord_type='x', axis=1)

# Calculate y coordinates of the line
lines_merc_gdf['y'] = lines_merc_gdf.apply(getLineCoords, geom='merc_coords', coord_type='y', axis=1)

# Let's see what we have now
lines_merc_gdf.head()

id name  objectid rt_symbol    shape_len  \
0  2000393.0    G     753.0         G  2438.200249   
1  2000394.0    G     754.0         G  3872.834411   
2  2000469.0    Q     755.0         N  1843.366331   
3  2000294.0    M     756.0         B  1919.559203   
4  2000296.0    M     757.0         B  2385.698536   

                                 url  \
0  http://web.mta.info/nyct/service/   
1  http://web.mta.info/nyct/service/   
2  http://web.mta.info/nyct/service/   
3  http://web.mta.info/nyct/service/   
4  http://web.mta.info/nyct/service/   

                                            geometry  \
0  LINESTRING (-73.99488 40.68020, -73.99427 40.6...   
1  LINESTRING (-73.97958 40.65993, -73.97966 40.6...   
2  LINESTRING (-73.97586 40.57597, -73.97654 40.5...   
3  LINESTRING (-73.92414 40.75229, -73.92405 40.7...   
4  LINESTRING (-73.91345 40.75617, -73.90905 40.7...   

                                         merc_coords  \
0  LINESTRING (-8237071.834 4965285.580, -8237004...   
1  LINESTRING (-8235368.666 4962310.184, -8235377...   
2  LINESTRING (-8234954.663 4949997.756, -8235031...   
3  LINESTRING (-8229198.018 4975873.004, -8229187...   
4  LINESTRING (-8228007.267 4976443.278, -8227517...   

                                                   x  \
0  [-8237071.833922522, -8237004.980569415, -8236...   
1  [-8235368.666198536, -8235377.54244116, -82354...   
2  [-8234954.662664757, -8235031.0160596855, -823...   
3  [-8229198.01779858, -8229187.532231273, -82291...   
4  [-8228007.26664242, -8227517.747062346, -82272...   

                                                   y  
0  [4965285.580068259, 4965374.625638519, 4965385...  
1  [4962310.183773335, 4962293.559848732, 4962153...  
2  [4949997.755558339, 4949966.0184754, 4949961.7...  
3  [4975873.004364504, 4975882.386061685, 4975895...  
4  [4976443.278149745, 4976139.561556996, 4975988...

We now have our mercator coordinates which we can use to plot the lines. Next, let's add some color to the lines!

First step is to identify the lines. The lines identified are in the 'name' column of "lines_merc_gdf". 

In [57]:
lines_merc_gdf['name'].unique()

array(['G', 'Q', 'M', 'S', 'A', 'B-D', 'B-D-F-M', 'R', 'N-Q-R', 'N-Q',
       'N-R', 'F', 'F-M', 'E', '7', 'J-Z', 'L', 'A-C', 'D', '1-2-3', 'B',
       '4-5-6', 'N', '1', 'N-W', '2-3', '2', '4-5', '5', '4', '3',
       'A-C-E', 'N-Q-R-W', 'N-R-W', '6', 'R-W'], dtype=object)

As you can see, multiple trains share the same lines, but the trains that share a line are assigned to the same color. For instance, if you read a NYC subway map, you will see that the 1, 2, 3 trains share a line but are assigned to a red color. 

We identified the hex codes for the subway line colors accessing https://www.color-hex.com/color-palettes/. However, there are many websites devoted to having a expansive listing of hex codes and numerous color palettes.  

In [ ]:
#Line Colors:
    #1,2,3 = #f20404
    #4,5,6 = #2fbf03
    #7 = #8c00a7
    #A,C,E = #024ed0
    #B,D,F,M = #f08b0e
    #G = #6be12f
    #J,Z = #ae5913
    #L = #9c9894
    #N,Q,R,W = #f6d50a


Once we have the hex codes assigned for the subway line colors, we'll build a "line_color" function, using numerous conditional statements. We have to make sure we have to include all lines and ensure we're assigning the appropriate hex codes. Once we've assigned the hex codes to the lines, we create a new column in "lines_merc_gdf" called color which will include the assigned hex codes.

In [ ]:
#Applying the appropriate colors (color of the lines that you see on NYC Subway Map) to the subway lines

In [50]:
def line_color(lines_merc_gdf):
    if(lines_merc_gdf['name']=='1') or (lines_merc_gdf['name']=='1-2-3') or (lines_merc_gdf['name']=='2-3') or (lines_merc_gdf['name']=='2') or (lines_merc_gdf['name']=='3'):
        return '#f20404'
    elif(lines_merc_gdf['name']=='4') or (lines_merc_gdf['name']=='5') or (lines_merc_gdf['name']=='6') or (lines_merc_gdf['name']=='4-5') or (lines_merc_gdf['name']=='4-5-6'):
        return '#2fbf03'
    elif(lines_merc_gdf['name']=='7'):
        return '#8c00a7'
    elif(lines_merc_gdf['name']=='A') or (lines_merc_gdf['name']=='A-C') or (lines_merc_gdf['name']=='E') or (lines_merc_gdf['name']=='A-C-E'):
        return '#024ed0'
    elif(lines_merc_gdf['name']=='B') or (lines_merc_gdf['name']=='D') or (lines_merc_gdf['name']=='B-D') or (lines_merc_gdf['name']=='F') or (lines_merc_gdf['name']=='M') or (lines_merc_gdf['name']=='F-M') or (lines_merc_gdf['name']=='B-D-F-M'):
        return '#f08b0e'
    elif(lines_merc_gdf['name']=='G'):
        return '#6be12f'
    elif(lines_merc_gdf['name']=='J-Z'):
        return '#ae5913'
    elif(lines_merc_gdf['name']=='L'):
        return '#9c9894'
    elif(lines_merc_gdf['name']=='S'):
        return '#5a5952'
    else:
        return '#f6d50a'   

In [51]:
lines_merc_gdf['color'] = lines_merc_gdf.apply(line_color, axis = 1)

In [52]:
# Make a copy and drop the geometry column
lines_merc_df = lines_merc_gdf.drop(['geometry','merc_coords'], axis=1).copy()

# Point DataSource
lines_merc_source = ColumnDataSource(lines_merc_df)

In [53]:
color = lines_merc_df['color']

Let's see how our map looks with the colors applied to the lines.

In [54]:
# Initialize our plot figure
p = figure(title="A map of the MTA New York City Subway System")

tile_provider = get_provider(Vendors.CARTODBPOSITRON)
p.add_tile(tile_provider)

# Add the lines to the map from our 'msource' ColumnDataSource -object
p.multi_line('x', 'y', source=lines_merc_source, color='color', line_width=2)

# Output filepath
output_file("NYC_Subway_System_Map.html")

# Save the map
show(p)

We have our colorful lines plotted on our map. Now it's time put incorporate station ridership data that we've created in the Part II into our map!

In [ ]:
###Putting it all together###

In [55]:
#Add stations and subway lines and coloring by ridership by station
source = ColumnDataSource(data=dict(
                        x=list(df['coords_x']), 
                        y=list(df['coords_y']),
                        y_2014 = list(df['y_2014']),
                        y_2015 = list(df['y_2015']),
                        y_2016 = list(df['y_2016']),
                        y_2017 = list(df['y_2017']),
                        y_2018 = list(df['y_2018']),
                        y_2019 = list(df['y_2019']),
                        Station = list(df['Station_Name']),
                        Subway_Lines=list(df['Subway_lines']),
                        Change=list(df['Chg']),
                        Pct_chg_2019=list(df['Pct_chg']),
                        Rank_2019=list(df['Rank_2019'])))

hover = HoverTool(tooltips=[
    ("Station", "@Station"),
    ("Subway Lines", "@Subway_Lines"),
    ("2019", "@{y_2019}{0,0}"),
    ("2018", "@{y_2018}{0,0}"),
    ("2017", "@{y_2017}{0,0}"),
    ("2016", "@{y_2016}{0,0}"),
    ("2015", "@{y_2015}{0,0}"),
    ("2014", "@{y_2014}{0,0}"),
    ("Actual change from 2018", "@Change"),
    ("% change from 2018", "@Pct_chg_2019"),
    ("Rank", "@Rank_2019")
])

p = figure(title = 'Average Weekday Subway Ridership, 2019',
           x_axis_type="mercator",
           y_axis_type="mercator",
           tools=[hover, 'wheel_zoom']
          )

#tile_provider = get_provider(Vendors.CARTODBPOSITRON)
#p.add_tile(tile_provider)

#Define a sequential multi-hue color palette.
#palette = brewer['Plasma'][8]
#palette = mpl['Plasma'][4]
#palette = Plasma[4]
palette = Magma[11]

#Reverse color order so that dark purple/magenta is to signal high activity.
palette = palette[::-1]

#exp_cmap = LinearColorMapper(palette=palette,
#                            low = min(df['y_2019']),
#                            high = max(df['y_2019']))

exp_cmap = EqHistColorMapper(palette=palette, low=min(df['y_2019']), high=max(df['y_2019']))

#p.circle(x='x',
#         y='y',
#         source=source,
#         size=6,
#         line_color="black",
#         fill_color={"field":"y_2019", "transform":exp_cmap},
#        alpha=1
#        )

bar = ColorBar(color_mapper=exp_cmap, location=(0,0))
#bar.formatter = NumeralTickFormatter(format'0,0')
p.add_layout(bar, 'left')


# Output filepath
#output_file("NYC_Subway_System_Map.html")

# Save the map
#show(p)

Our final product!!!

In [64]:
# Initialize our plot figure
p = figure(title="MTA New York City Subway System and Pre-pandemic Average Weekday Ridership",
          tools=[hover,'wheel_zoom', 'pan'])

tile_provider = get_provider(Vendors.CARTODBPOSITRON)
p.add_tile(tile_provider)

# Add the lines to the map from our 'msource' ColumnDataSource -object
p.multi_line('x', 'y', source=lines_merc_source, color='color', line_width=3)

p.circle(x='x',
         y='y',
         source=source,
         size=6,
         line_color="black",
         fill_color={"field":"y_2019", "transform":exp_cmap},
         alpha=1
        )
p.add_layout(bar, 'right')


show(p)

In [65]:
#line_list = ['1','2','3','2-3','1-2-3','4','5','6','4-5','4-5-6','7','A','A-C','A-C-E','B','D','B-D','F','M','F-M',
#            'B-D-F-M','G','J-Z','L','S','N','R','Q','W','R-W','N-Q','N-R-W','N-Q-R-W','N-W']

In [ ]:
### END OF FILE ###